# XGBoost with Cross Validation and PCA

In [ ]:
import numpy as np
import pandas as pd
from numba import njit
import vectorbtpro as vbt
vbt.settings.set_theme("dark")
vbt.settings.plotting["layout"]["width"] = 800
vbt.settings.plotting['layout']['height'] = 200
import warnings
warnings.filterwarnings("ignore")
import matplotlib.pyplot as plt
from collections import Counter


from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.linear_model import Ridge, Lasso, ElasticNet, LinearRegression, LogisticRegression
from sklearn.svm import SVR, SVC
from xgboost import XGBRegressor
from xgboost import plot_importance
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestClassifier
clf = RandomForestClassifier(random_state=42) # random forest classifier
from joblib import dump, load

from sklearn.cluster import KMeans
from sklearn.model_selection import cross_val_score, KFold
import matplotlib.pyplot as plt


### Modeling
The class Splitter can also be helpful in cross-validating ML models. In particular, you can casually step upon a class SKLSplitter that acts as a regular cross-validator from scikit-learn by subclassing BaseCrossValidator. We'll demonstrate its usage on a simple classification problem of predicting the best entry and exit timings.

Before we start, we need to decide on features and labels that should act as predictor and response variables respectively. Features are usually multi-columnar time-series DataFrames where each row contains multiple data points (one per column) that should predict the same row in labels. Labels are usually a single-columnar time-series Series that should be predicted. Ask yourself the following questions to easily come up with a decision:

"How can the future performance be represented, preferably as a single number? Should it be the price at the next bar, the average price change over the next week, a vector of weights for rebalancing, a boolean containing a signal, or something else?"
"What kind of data that encompasses the past performance is likely to predict the future performance? Should it be indicators, news sentiment index, past backtesting results, or something else?"
"Which ML model can handle such a task?" (remember that most models are limited to just a couple of specific feature and label formats!)
For the sake of an example, we'll fit a random forest classifier on all TA-Lib indicators stacked along columns to predict the binary labels generated by the label generator TRENDLB, where 1 means an uptrend and 0 means a downtrend. Sounds like fun 😌

Build a pipeline to impute and (standard-)normalize the data, [reduce the dimensionality](https://scikit-learn.org/stable/auto_examples/compose/plot_digits_pipe.html) of the features, as well as fit one of the [linear](https://scikit-learn.org/stable/modules/linear_model.html) models to predict the average price change over the next n bars (i.e., regression task!). Based on each prediction, you can then decide whether a position is worth opening or closing out. 

# Helper functions
Create dollar bars and add them to the original df

In [ ]:

def dollar_bar_func(ohlc_df, dollar_bar_size):
    # Calculate dollar value traded for each row
    ohlc_df['DollarValue'] = ohlc_df['Close'] * ohlc_df['Volume']
    
    # Calculate cumulative dollar value
    ohlc_df['CumulativeDollarValue'] = ohlc_df['DollarValue'].cumsum()
    
    # Determine the number of dollar bars
    num_bars = int(ohlc_df['CumulativeDollarValue'].iloc[-1] / dollar_bar_size)
    
    # Generate index positions for dollar bars
    bar_indices = [0]
    cumulative_value = 0
    for i in range(1, len(ohlc_df)):
        cumulative_value += ohlc_df['DollarValue'].iloc[i]
        if cumulative_value >= dollar_bar_size:
            bar_indices.append(i)
            cumulative_value = 0
    
    # Create a new dataframe with dollar bars
    dollar_bars = []
    for i in range(len(bar_indices) - 1):
        start_idx = bar_indices[i]
        end_idx = bar_indices[i + 1]
        
        dollar_bar = {
            'Open': ohlc_df['Open'].iloc[start_idx],
            'High': ohlc_df['High'].iloc[start_idx:end_idx].max(),
            'Low': ohlc_df['Low'].iloc[start_idx:end_idx].min(),
            'Close': ohlc_df['Close'].iloc[end_idx-1],
            'Volume': ohlc_df['Volume'].iloc[start_idx:end_idx].sum(),
            'Quote volume': ohlc_df['Quote volume'].iloc[start_idx:end_idx].sum(),
            'Trade count': ohlc_df['Trade count'].iloc[start_idx:end_idx].sum(),
            'Taker base volume': ohlc_df['Taker base volume'].iloc[start_idx:end_idx].sum(),
            'Taker quote volume': ohlc_df['Taker quote volume'].iloc[start_idx:end_idx].sum()
        }
        
        if isinstance(ohlc_df.index, pd.DatetimeIndex):
            dollar_bar['Open Time'] = ohlc_df.index[start_idx]
            dollar_bar['Close Time'] = ohlc_df.index[end_idx-1] - pd.Timedelta(milliseconds=1)
        elif 'Open Time' in ohlc_df.columns:
            dollar_bar['Open Time'] = ohlc_df['Open Time'].iloc[start_idx]
            dollar_bar['Close Time'] = ohlc_df['Open Time'].iloc[end_idx] - pd.Timedelta(milliseconds=1)
        
        dollar_bars.append(dollar_bar)
    
    dollar_bars_df = pd.concat([pd.DataFrame([bar]) for bar in dollar_bars], ignore_index=True)
    
    return dollar_bars_df

# Create a simple function to simplify the number so we can use it in our column names
def simplify_number(num):
    """
    Simplifies a large number by converting it to a shorter representation with a suffix (K, M, B).
    simplify_number(1000) -> 1K
    """
    suffixes = ['', 'K', 'M', 'B']
    suffix_index = 0

    while abs(num) >= 1000 and suffix_index < len(suffixes) - 1:
        num /= 1000.0
        suffix_index += 1

    suffix = suffixes[suffix_index] if suffix_index > 0 else ''
    simplified_num = f'{int(num)}{suffix}'

    return simplified_num

def merge_and_fill_dollar_bars(original_df, dollar_bars_df, dollar_bar_size):
    # Add prefix to column names in dollar bars dataframe
    dollar_bar_prefix = f'db_{simplify_number(dollar_bar_size)}_'
    dollar_bars_df_renamed = dollar_bars_df.add_prefix(dollar_bar_prefix)

    # Convert 'Open Time' columns to pandas datetime format and set them as index
    dollar_bars_df_renamed.index = pd.to_datetime(dollar_bars_df_renamed[dollar_bar_prefix + 'Open Time'])

    # Merge the dataframes on the index
    merged_df = original_df.merge(dollar_bars_df_renamed, how='left', left_index=True, right_index=True)

    # Set the flag for a new dollar bar with prefix
    merged_df[dollar_bar_prefix + 'NewDBFlag'] = ~merged_df[dollar_bar_prefix + 'Close'].isna()

    # Forward fill the NaN values for all columns except the new dollar bar flag
    columns_to_ffill = [col for col in merged_df.columns if col != dollar_bar_prefix + 'NewDBFlag']
    merged_df[columns_to_ffill] = merged_df[columns_to_ffill].fillna(method='ffill')

    # Fill the remaining NaN values in the new dollar bar flag column with False
    merged_df[dollar_bar_prefix + 'NewDBFlag'] = merged_df[dollar_bar_prefix + 'NewDBFlag'].fillna(False)
    
    # Assign the renamed 'Open Time' column back to the dataframe
    merged_df[dollar_bar_prefix + 'Open Time'] = merged_df[dollar_bar_prefix + 'Open Time']

    return merged_df





# Calculate Dollar Bars
Calc Dollar bars and then add technical analysis features

Uncomment this section if you want to run different size dollar bars

In [ ]:
# futures_1m = vbt.BinanceData.load('/Users/ericervin/Documents/Coding/data-repository/data/BTCUSDT_1m_futures.pkl')
# futures_1m_df = futures_1m.get()

In [ ]:
# dollar_bar_size = 90_000_000
# btc_dollar_bars = dollar_bar_func(futures_1m_df, dollar_bar_size=dollar_bar_size)
# btc_dollar_bars.index = pd.to_datetime(btc_dollar_bars['Open Time'])
# btc_dollar_bars.shape

In [ ]:
# Convert the dataframe back into a vbt data object
# btc_90M_db_vbt = vbt.BinanceData.from_data(btc_dollar_bars)


In [ ]:
# Save the dollarbars to a pickle file
# btc_90M_db_vbt.save('data/btc_90M_db_vbt.pkl')

# Load the dollar bars from pickle file
Take a small slice of the data for train/testing and leave some to be out of sample

In [ ]:
# List all the ta features we can use
print(dir(vbt.indicators))



In [ ]:
btc_90M_db_vbt = vbt.BinanceData.load('data/btc_90M_db_vbt.pkl')

data = btc_90M_db_vbt['2021-01-01':'2023-01-01']
outofsample_data = btc_90M_db_vbt['2023-01-01':'2023-06-03']
print(data.shape)
print(outofsample_data.shape)
# Wherever you saved the pickle file
data_path = 'data/BTCUSDT_1m.pkl'
# min_data = vbt.BinanceData.load(data_path).get()

In [ ]:
vbt.phelp(vbt.SUPERTREND.run)

In [ ]:
data.get()

In [ ]:
supertrend = data.run("supertrend").supert
supertrend

# Create the functions for the notebook

In [ ]:
def prepare_data(data, pivot_up_th=0.10, pivot_down_th=0.10, periods_future=150, drop_cols=['Open Time', 'Close Time']):
    """
    This function prepares the data for training the model.
    
    Parameters:
    data (DataFrame): The original DataFrame containing the data.
    pivot_up_th (float): Threshold for upward trend.
    pivot_down_th (float): Threshold for downward trend.
    periods_future (int): Number of periods in the future to predict.
    drop_cols (list): Columns to be dropped from the original DataFrame.
    
    Returns:
    X (DataFrame): The feature matrix.
    y (Series): The target vector.
    """
    pivot_up_th2 = pivot_up_th * 1.5
    pivot_down_th2 = pivot_down_th * 1.5
    pivot_up_th3 = pivot_up_th * 1.5
    pivot_down_th3 = pivot_down_th * 1.5
    lookback_window = 14*150 # Number of dollar bars we are predicting into the future times the typical RSI lookback window of 14
    
    # Generate the features (X) and target (y) dataframes
    X = data.get()
    # Set up multiple pivot trend labels
    pivot_info = data.run("pivotinfo", up_th=pivot_up_th, down_th=pivot_down_th)
    binary_pivot_labels = np.where(data.close > pivot_info.conf_value,1,0) # Create binary labels for pivot points
    X['trend'] = binary_pivot_labels # add pivot label as a feature
    
    pivot_info2 = data.run("pivotinfo", up_th=pivot_up_th2, down_th=pivot_down_th2)
    binary_pivot_labels2 = np.where(data.close > pivot_info2.conf_value,1,0) # Create binary labels for pivot points
    X['trend2'] = binary_pivot_labels2 # add pivot label as a feature
    
    pivot_info3 = data.run("pivotinfo", up_th=pivot_up_th3, down_th=pivot_down_th3)
    binary_pivot_labels3 = np.where(data.close > pivot_info3.conf_value,1,0) # Create binary labels for pivot points
    X['trend3'] = binary_pivot_labels3 # add pivot label as a feature
    
    # Add some TA features
    X['supert'] = data.run("supertrend").supert
    # X['superd'] = data.run("supertrend").superd
    # X['superl'] = data.run("supertrend").superl
    # X['supers'] = data.run("supertrend").supers
    X['vwap'] = data.run("VWAP").vwap
    X['rsi'] = data.run("rsi", window=lookback_window).rsi
    X['rsi_overbought'] = pd.Series(np.where(X['rsi'] > 70, 1, 0), index=X.index)
    X['rsi_oversold'] = pd.Series(np.where(X['rsi'] < 30, 1, 0), index=X.index)
    X['bb_width'] = data.run("bbands", window=lookback_window).bandwidth
    X['bb_width_pct'] = data.run("bbands", window=lookback_window).percent_b
    X['fast_k'] = data.run("stoch", fast_k_window=lookback_window, slow_k_window=50*14, slow_d_window=50*14).fast_k
    X['slow_k'] = data.run("stoch", fast_k_window=lookback_window, slow_k_window=50*14, slow_d_window=50*14).slow_k
    X['slow_k_trending_up'] = X['slow_k'] > X['slow_k'].shift(1)
    X['slow_k_trending_down'] = X['slow_k'] < X['slow_k'].shift(1)
    X['slow_d'] = data.run("stoch", fast_k_window=lookback_window, slow_k_window=50*14, slow_d_window=50*14).slow_d
    X['slow_k_over_slow_d'] = X['slow_k'] > X['slow_d']
    X['slow_k_under_slow_d'] = X['slow_k'] < X['slow_d']
    
    
    # Add in historical returns
    X['pct_change_20'] = data.close.pct_change(20)
    X['pct_change_40'] = data.close.pct_change(40)
    X['pct_change_60'] = data.close.pct_change(60)
    X['pct_change_100'] = data.close.pct_change(100)
    X['pct_change_160'] = data.close.pct_change(160)
    X['pct_change_260'] = data.close.pct_change(260)
    X['pct_change_420'] = data.close.pct_change(420)
    
    X['mid_range_momentum']= pd.Series(np.where(X['pct_change_100'] > X['pct_change_420'], 1, 0), index=X.index)
    X['short_range_momentum']= pd.Series(np.where(X['pct_change_20'] > X['pct_change_40'], 1, 0), index=X.index)
    X['short_over_long_momentum'] = pd.Series(np.where(X['pct_change_20'] > X['pct_change_420'], 1, 0), index=X.index)
    X['momentum_trending'] = pd.Series(np.where(X['pct_change_20'] > X['pct_change_20'].shift(1), 1, 0), index=X.index)
    
    # Drop the time columns
    X = X.drop(drop_cols, axis=1)
    # Add time features
    X['dayofmonth']  = X.index.day
    X['month']       = X.index.month
    X['year']        = X.index.year
    X['hour']        = X.index.hour
    X['minute']      = X.index.minute
    X['dayofweek']   = X.index.dayofweek
    # X['dayofyear']   = X.index.dayofyear

    # Now we are trying to generate future price predictions so we will set the y labels to the price change n periods in the future
    y = (data.close.shift(-periods_future) / data.close - 1).rolling(periods_future).mean() # future price change we use rolling mean to smooth the data

    # Preprocessing steps to handle NaNs
    X = X.replace([-np.inf, np.inf], np.nan) # replace inf with nan
    invalid_column_mask = X.isnull().all(axis=0)
    X = X.loc[:, ~invalid_column_mask] # drop invalid columns
    invalid_row_mask = X.isnull().any(axis=1) | y.isnull() # drop rows that have nan in any column or in y

    # Drop invalid rows in X and y
    X = X.loc[~invalid_row_mask]
    y = y.loc[~invalid_row_mask]

    return X, y

def create_pipeline(X, model='xgb'):
    """
    Create a scikit-learn pipeline.

    Parameters:
    model (str): The model to use in the pipeline. Default is 'xgb' (XGBoost).

    Returns:
    pipeline (Pipeline): The scikit-learn pipeline.
    """
    X_shape = X.shape
    # Construct the pipeline
    steps = [
        ('imputation', SimpleImputer(strategy='mean')),  # Imputation replaces missing values
        ('scaler', StandardScaler()),  # StandardScaler normalizes the data
        # ('pca', PCA(n_components=15))  # PCA reduces dimensionality
    ]

    if model == 'xgb':
        steps.append(('model', XGBRegressor(objective='reg:squarederror')))  # XGBoost regression is used as the prediction model
    elif model == 'ridge':
        steps.append(('model', Ridge()))  # Ridge regression
    elif model == 'linear':
        steps.append(('model', LinearRegression()))  # Linear regression
    elif model == 'logistic':
        steps.append(('model', LogisticRegression()))  # Logistic regression
    elif model == 'lasso':
        steps.append(('model', Lasso()))  # Lasso regression
    elif model == 'elasticnet':
        steps.append(('model', ElasticNet()))  # ElasticNet regression
    elif model == 'svr':
        steps.append(('model', SVR()))  # Support Vector Regression
    else:
        raise ValueError("Invalid model name. Choose from 'xgb', 'ridge', 'linear', 'logistic', 'lasso', 'elasticnet', 'svr'.")

    pipeline = Pipeline(steps)
    
    return pipeline

def create_cv(X, min_length=600, offset=200, split=-200, set_labels=["train", "test"]):
    """
    Create a cross-validation splitter.

    Parameters:
    X (DataFrame): The feature matrix.
    min_length (int): The minimum length of a sample for cross-validation.
    offset (int): The offset used in cross-validation splitting.
    split (int): Index at which to split the data in cross-validation.
    set_labels (list): Labels for the train and test sets in cross-validation.

    Returns:
    cv_splitter (SKLSplitter): The cross-validation splits created from cv.get_splitter(X).
    cv (SKLSplitter): The cross-validation object.
    """

    # Cross-validate Creates a cross-validation object with all the indexes for each cv split
    cv = vbt.SKLSplitter("from_expanding", min_length=min_length, offset=offset, split=split, set_labels=set_labels)
    cv_splitter = cv.get_splitter(X)
    
    return cv_splitter, cv

def create_cv_with_gap(X, min_length=600, test_amount=200, gap = 150, set_labels=["train", "test"]):
    """
    Create a cross-validation splitter.

    Parameters:
    X (DataFrame): The feature matrix.
    min_length (int): The minimum length of a sample for cross-validation.
    offset (int): The offset used in cross-validation splitting.
    split (int): Index at which to split the data in cross-validation.
    set_labels (list): Labels for the train and test sets in cross-validation.

    Returns:
    cv_splitter (SKLSplitter): The cross-validation splits created from cv.get_splitter(X).
    cv (SKLSplitter): The cross-validation object.
    """

    # Cross-validate Creates a cross-validation object with all the indexes for each cv split
    cv = vbt.SKLSplitter("from_expanding", 
                         min_length=min_length, 
                         offset=test_amount, 
                         split=(1.0, vbt.RelRange(length=gap, is_gap=True), test_amount), 
                         set_labels=set_labels,
                         split_range_kwargs=dict(backwards=True)
                         )
    cv_splitter = cv.get_splitter(X)
    
    return cv_splitter, cv

def create_rolling_cv(X, length=2000, split=0.90, offset=True, offsetlen=0, set_labels=["train", "test"]):
    """
    Create a cross-validation splitter.

    Parameters:
    X (DataFrame): The feature matrix.
    min_length (int): The minimum length of a sample for cross-validation.
    split (float): percent of window to split training vs testing.
    set_labels (list): Labels for the train and test sets in cross-validation.
    offset (bool): Whether to offset the splits, True shifts the window forward by only the test number.

    Returns:
    cv_splitter (SKLSplitter): The cross-validation splits created from cv.get_splitter(X).
    cv (SKLSplitter): The cross-validation object.
    """
    if offset:
        offsetlen = 2*(length * split) - length
        cv = vbt.SKLSplitter("from_rolling", length=length, split=split, offset=-offsetlen, offset_anchor="prev_end", set_labels=set_labels)
        cv_splitter = cv.get_splitter(X) 
        return cv_splitter, cv
    # Cross-validate Creates a cross-validation object with all the indexes for each cv split
    else:
        cv = vbt.SKLSplitter("from_rolling", length=length, split=split, set_labels=set_labels) # offset=-offsetlen, offset_anchor="prev_end",
        cv_splitter = cv.get_splitter(X) 
        return cv_splitter, cv
    

def create_rolling_cv_with_gap(X, length=500, split=0.70, gap=150, set_labels=["train", "test"]):
    """
    Create a cross-validation splitter.

    Parameters:
    X (DataFrame): The feature matrix.
    length (int): The length of a sample for cross-validation.
    split (float): The percent of the sample to use for training.
    gap (int): The gap between the training and test sets.
    set_labels (list): Labels for the train and test sets in cross-validation.

    Returns:
    cv_splitter (SKLSplitter): The cross-validation splits created from cv.get_splitter(X).
    cv (SKLSplitter): The cross-validation object.
    """
    assert length > gap, "Length must be greater than gap"

    split_size = int((length - gap) * split) # Total length of the set minus the gap times the split percent is training set
    test_size = length - split_size - gap # Total length minus the training set minus the gap is the test set
    offset = -(split_size-test_size) # Offset the split by the difference between the training and test set this gets the next test set to start where the last one ended

    cv = vbt.SKLSplitter("from_rolling", 
                        length=length,
                        split=(split_size, vbt.RelRange(length=gap, is_gap=True), 1.0),
                        offset=offset,
                        set_labels=set_labels)
    cv_splitter = cv.get_splitter(X)
    return cv_splitter, cv
    

def cross_validate_and_train(pipeline, X, y, cv_splitter, model_name="", verbose_interval=10, n_clusters=6):
    # Predictions
    X_slices = cv_splitter.take(X)
    y_slices = cv_splitter.take(y)
    
    # Print total number of splits
    total_splits = len(X_slices.index.unique(level="split"))
    print(f"Total number of cross-validation splits: {total_splits}")


    test_labels = []
    test_preds = []
    for split in X_slices.index.unique(level="split"):  
        X_train_slice= X_slices[(split, "train")]  
        y_train_slice= y_slices[(split, "train")] 
        X_test_slice = X_slices[(split, "test")]
        y_test_slice = y_slices[(split, "test")]
        
        # Fit the pipeline on the training data
        pipeline.fit(X_train_slice, y_train_slice)
        
        # Fit the KMeans clustering algorithm on the training data using only the original columns in X_slices
        kmeans = KMeans(n_clusters=n_clusters, random_state=0)
        kmeans.fit(X_train_slice)
        
        # Get the cluster labels for the training data using the KMeans clustering algorithm fitted on the training data
        train_cluster_labels = kmeans.predict(X_train_slice)

        # Add the "cluster" column to the training data using the cluster labels obtained above
        train_data = X_train_slice.copy()
        train_data["cluster"] = train_cluster_labels
        # Get the cluster labels for the test data using the KMeans clustering algorithm fitted on the training data
        test_cluster_labels = kmeans.predict(X_test_slice) #TODO: use the kmeans model fitted on the training data to predict the cluster labels for the test data
        # Get the cluster labels and their counts for the test data
        test_cluster_counts = Counter(test_cluster_labels)
        # Add the "cluster" column to the test data using the cluster labels obtained above
        test_data = X_test_slice.copy()
        test_data["cluster"] = test_cluster_labels
        # Fit the pipeline on the training data
        pipeline.fit(train_data, y_train_slice)
        
        # Make predictions on the test data
        test_pred = pipeline.predict(test_data)  
        test_pred = pd.Series(test_pred, index=y_test_slice.index)
        test_labels.append(y_test_slice)
        test_preds.append(test_pred)

        # Only print the MSE every 'verbose_interval' splits
        if split % verbose_interval == 0:
            print(f"{model_name} Split {split} Mean Squared Error: {mean_squared_error(y_test_slice, test_pred)}")
            # Print the cluster labels and their counts
            print(f"Cluster Sizes:")
            for label, count in test_cluster_counts.items():
                print(f"Cluster {label}: {count}")

    # Concatenate the test labels and predictions into a single Series
    test_labels = pd.concat(test_labels).rename("labels")  
    test_preds = pd.concat(test_preds).rename("preds")
    
    # Drop Duplicates
    test_labels = test_labels[~test_labels.index.duplicated(keep='first')]
    test_preds = test_preds[~test_preds.index.duplicated(keep='first')]
    
    return pipeline, test_labels, test_preds

def evaluate_predictions(test_labels, test_preds, model_name=""):
    # Show the accuracy of the predictions
    mse = mean_squared_error(test_labels, test_preds)
    rmse = np.sqrt(mse)  # or use mean_squared_error with squared=False
    mae = mean_absolute_error(test_labels, test_preds)
    r2 = r2_score(test_labels, test_preds)

    print(f"{model_name} Mean Squared Error (MSE): {mse}")
    print(f"{model_name} Root Mean Squared Error (RMSE): {rmse}")
    print(f"{model_name} Mean Absolute Error (MAE): {mae}")
    print(f"{model_name} R-squared: {r2}")

    return mse, rmse, mae, r2

def extract_feature_importance(pipeline, X):
    fitted_model = pipeline.named_steps['model']
    feature_names = X.columns.tolist()
    feature_names.append('cluster')

    # Create a DataFrame using a Dictionary
    data = {'feature_names': feature_names, 'feature_importance': fitted_model.feature_importances_}
    print(len(feature_names))
    print(len(fitted_model.feature_importances_))
    fi_df = pd.DataFrame(data)

    # Sort the DataFrame in order decreasing feature importance
    fi_df.sort_values(by=['feature_importance'], ascending=False, inplace=True)

    # Define size of bar plot
    plt.figure(figsize=(12,8))
    # Plot bar chart
    plt.barh(fi_df['feature_names'], fi_df['feature_importance'], align='center')
    # Add chart labels
    plt.xlabel('FEATURE IMPORTANCE')
    plt.ylabel('FEATURE NAMES')
    plt.show()

    # Print feature names and importance
    for index, row in fi_df.iterrows():
        print(f"{row['feature_names']}: {row['feature_importance']}")
        
def plot_prediction_vs_actual(x, y, title='Test Predictions vs Actual Results'):
    """
    Plots predictions against actual results and calculates the line of best fit.

    Parameters:
    x (Series): Predictions
    y (Series): Actual results
    title (str): Title of the plot

    Returns:
    None
    """

    # Create condition masks for different data types
    tp_condition = (x > 0) & (y > 0)  # True positives condition
    tn_condition = (x < 0) & (y < 0)  # True negatives condition
    fp_condition = (x > 0) & (y < 0)  # False positives condition
    fn_condition = (x < 0) & (y > 0)  # False negatives condition

    # Calculate percent in each condition
    tp_percent = (tp_condition.sum() / len(x)) * 100
    tn_percent = (tn_condition.sum() / len(x)) * 100
    fp_percent = (fp_condition.sum() / len(x)) * 100
    fn_percent = (fn_condition.sum() / len(x)) * 100

    # Create scatter plots for each condition with different colors
    plt.scatter(x[tp_condition], y[tp_condition], color='green', alpha=0.2, label='True Positives')
    plt.scatter(x[tn_condition], y[tn_condition], color='pink', alpha=0.2, label='True Negatives')
    plt.scatter(x[fp_condition], y[fp_condition], color='blue', alpha=0.2, label='False Positives')
    plt.scatter(x[fn_condition], y[fn_condition], color='blue', alpha=0.2, label='False Negatives')

    # Calculate the line of best fit
    coefficients = np.polyfit(x, y, 1)
    polynomial = np.poly1d(coefficients)

    # Generate y-values based on the polynomial
    y_fit = polynomial(x)

    # Plot the line of best fit
    plt.plot(x, y_fit, color='red', label='Line of Best Fit')

    # Add title and labels to the axes
    plt.title(title)
    plt.xlabel('Predictions')
    plt.ylabel('Actual Results')

    # Add a legend
    plt.legend()

    # Print the equation of the line
    slope, intercept = coefficients
    print(f"The equation of the regression line is: y = {slope:.3f}x + {intercept:.3f}")

    # Alternatively, you can display the equation on the plot
    plt.text(0.05, 0.95, f'y = {slope:.3f}x + {intercept:.3f}', transform=plt.gca().transAxes)

    # Print the percent in each quadrant
    print(f"\nPercentage of True Positives: {tp_percent:.2f}%")
    print(f"Percentage of True Negatives: {tn_percent:.2f}%")
    print(f"Percentage of False Positives: {fp_percent:.2f}%")
    print(f"Percentage of False Negatives: {fn_percent:.2f}%")

    # Show the plot
    plt.show()


    
# Simulate a portfolio making trades based on predictions
def simulate_pf(data, test_preds, open_long_th=0.01, close_long_th=0.0, close_short_th=0.0, open_short_th=-0.01, plot=True):
    insample_pf = vbt.Portfolio.from_signals(
    data.close[test_preds.index],  # use only the test set
    entries         = test_preds > open_long_th,  # long when probability of price increase is greater than 2%
    exits           = test_preds < close_long_th,  # long when probability of price increase is greater than 2%
    short_entries   = test_preds < open_short_th,  # long when probability of price increase is greater than 2%
    short_exits     = test_preds > close_short_th,  # short when probability prediction is less than -5%
    # direction="both" # long and short
)
    print(insample_pf.stats())
    if plot==True:
        insample_pf.plot().show()
    return insample_pf

In [ ]:
# Future Prediction Length
periods_future = 150
pivot_up_th = 0.01
pivot_down_th = 0.01

# Establish the training and testing sets for cross validation
windowsize = 10000
split = 0.90
gap = periods_future

# Prep Data
X, y = prepare_data(data, pivot_up_th=pivot_up_th, pivot_down_th=pivot_down_th, periods_future=periods_future) # in-sample
# print(X)
Xoos, yoos = prepare_data(outofsample_data, pivot_up_th=pivot_up_th, pivot_down_th=pivot_down_th, periods_future=periods_future) # out-of-sample
# print(X.columns)
# Set up the pipeline and create the cross validation splits
pipeline = create_pipeline(X, model='xgb')
cv_splitter, cv = create_rolling_cv_with_gap(X, length=windowsize, split=split, gap=periods_future, set_labels=["train", "test"])

# Train and cross-validate
final_pipeline, test_labels, test_preds = cross_validate_and_train(pipeline, X, y, cv_splitter, model_name="In-Sample", verbose_interval=50)

# Evaluate
mse, rmse, mae, r2 = evaluate_predictions(test_labels, test_preds, model_name="In-Sample")

# Check the scatter plot of predictions vs actual results
plot_prediction_vs_actual(test_preds, test_labels, title='In-Sample Predictions vs Actual Results')
# Check the primary features that impacted the model
extract_feature_importance(final_pipeline, X)

# Show heatmap of the predictions ontop of a price plot
# data.close.vbt.overlay_with_heatmap(test_preds).show_svg()

# Simulate a portfolio
insample_pf = simulate_pf(data=data, test_preds=test_preds, open_long_th=0.03, close_long_th=0.0, close_short_th=-0.03, open_short_th=-0.01, plot=False)


Do the exact same thing but do it with expanding cross validation so the training set gets bigger and bigger

In [ ]:
# Future Prediction Length
periods_future = 150
pivot_up_th = 0.10
pivot_down_th = 0.05

# Establish the training and testing sets for cross validation
windowsize = 10000
split = 0.90
test_amount = int(split * windowsize)
gap = periods_future

# Prep Data
X, y = prepare_data(data, pivot_up_th=pivot_up_th, pivot_down_th=pivot_down_th, periods_future=periods_future) # in-sample
Xoos, yoos = prepare_data(outofsample_data, pivot_up_th=pivot_up_th, pivot_down_th=pivot_down_th, periods_future=periods_future) # out-of-sample

# Set up the pipeline and create the cross validation splits
pipeline = create_pipeline(X, model='xgb')
cv_splitter, cv = create_cv_with_gap(X, min_length=windowsize, test_amount=test_amount, gap=periods_future, set_labels=["train", "test"])

# Train and cross-validate
final_pipeline, test_labels, test_preds = cross_validate_and_train(pipeline, X, y, cv_splitter, model_name="In-Sample", verbose_interval=50)

# Evaluate
mse, rmse, mae, r2 = evaluate_predictions(test_labels, test_preds, model_name="In-Sample")

# Check the scatter plot of predictions vs actual results
plot_prediction_vs_actual(test_preds, test_labels, title='In-Sample Predictions vs Actual Results')
# Check the primary features that impacted the model
extract_feature_importance(final_pipeline, X)

# Show heatmap of the predictions ontop of a price plot
# data.close.vbt.overlay_with_heatmap(test_preds).show_svg()

# Simulate a portfolio
insample_pf = simulate_pf(data=data, test_preds=test_preds, open_long_th=0.10, close_long_th=0.0, close_short_th=-0.0, open_short_th=-0.05, plot=False)


In [ ]:
def plot_prediction_vs_actual(x, y, title='Test Predictions vs Actual Results'):
    """
    Plots predictions against actual results and calculates the line of best fit.

    Parameters:
    x (Series): Predictions
    y (Series): Actual results
    title (str): Title of the plot

    Returns:
    None
    """

    # Create condition masks for different data types
    tp_condition = (x > 0) & (y > 0)  # True positives condition
    tn_condition = (x < 0) & (y < 0)  # True negatives condition
    fp_condition = (x > 0) & (y < 0)  # False positives condition
    fn_condition = (x < 0) & (y > 0)  # False negatives condition

    # Calculate percent in each condition
    tp_percent = (tp_condition.sum() / len(x)) * 100
    tn_percent = (tn_condition.sum() / len(x)) * 100
    fp_percent = (fp_condition.sum() / len(x)) * 100
    fn_percent = (fn_condition.sum() / len(x)) * 100

    # Create scatter plots for each condition with different colors
    plt.scatter(x[tp_condition], y[tp_condition], color='green', alpha=0.2, label='True Positives')
    plt.scatter(x[tn_condition], y[tn_condition], color='pink', alpha=0.2, label='True Negatives')
    plt.scatter(x[fp_condition], y[fp_condition], color='blue', alpha=0.2, label='False Positives')
    plt.scatter(x[fn_condition], y[fn_condition], color='blue', alpha=0.2, label='False Negatives')

    # Calculate the line of best fit
    coefficients = np.polyfit(x, y, 1)
    polynomial = np.poly1d(coefficients)

    # Generate y-values based on the polynomial
    y_fit = polynomial(x)

    # Plot the line of best fit
    plt.plot(x, y_fit, color='red', label='Line of Best Fit')

    # Add title and labels to the axes
    plt.title(title)
    plt.xlabel('Predictions')
    plt.ylabel('Actual Results')

    # Add a legend
    plt.legend()

    # Print the equation of the line
    slope, intercept = coefficients
    print(f"The equation of the regression line is: y = {slope:.3f}x + {intercept:.3f}")

    # Alternatively, you can display the equation on the plot
    plt.text(0.05, 0.95, f'y = {slope:.3f}x + {intercept:.3f}', transform=plt.gca().transAxes)

    # Print the percent in each quadrant
    print(f"\nPercentage of True Positives: {tp_percent:.2f}%")
    print(f"Percentage of True Negatives: {tn_percent:.2f}%")
    print(f"Percentage of False Positives: {fp_percent:.2f}%")
    print(f"Percentage of False Negatives: {fn_percent:.2f}%")

    # Show the plot
    plt.show()


In [ ]:
plot_prediction_vs_actual(test_preds, test_labels, title='In-Sample Predictions vs Actual Results')

In [ ]:
print(test_preds)
# check for duplicates in index
print(test_preds.index.duplicated().sum())

If you want to look at different portfolio simulations you can change the thresholds for opening and closing trades

In [ ]:
alt_param_pf = simulate_pf(
    data,
    test_preds,
    open_long_th    =0.10,
    close_long_th   =0.0,
    close_short_th  =-0.05,
    open_short_th   =-0.08,
    plot=False,
)  # The _ is because we don't need return the portfolio object we are just looking for the output
alt_param_pf.plot().show()

In [ ]:
extract_feature_importance(final_pipeline, X)

In [ ]:


# Combine the test predictions and labels into a single array
test_data = np.column_stack((test_preds, test_labels))
n_clusters = 12
# Run KMeans clustering with 6 clusters
kmeans = KMeans(n_clusters=n_clusters, random_state=0)

# Create a pipeline with all the data
pipeline_all = create_pipeline(X, model='xgb')

# Evaluate the pipeline with cross-validation
scores_all = cross_val_score(pipeline_all, X, y, cv=50, scoring='neg_mean_squared_error')
print(f'All data scores: {scores_all}')

# Evaluate the pipeline with cross-validation using the clustering step
scores_filtered = []
for train_index, test_index in KFold(n_splits=50).split(X):
    print(f'Fold {len(scores_filtered) + 1}')
    # Fit the clustering algorithm on the training data
    kmeans.fit(np.column_stack((y[train_index], pipeline_all.predict(X[train_index]).reshape(-1, 1))))

    # Get the cluster labels for the test data
    cluster_labels = kmeans.predict(np.column_stack((y[test_index], pipeline_all.predict(X[test_index]).reshape(-1, 1))))

    # Filter out the data points in the first cluster
    filtered_data = X[test_index][cluster_labels != 0]
    filtered_labels = y[test_index][cluster_labels != 0]

    # Evaluate the pipeline on the filtered data
    score = pipeline_all.score(filtered_data, filtered_labels)
    scores_filtered.append(score)

print(f'Filtered data scores: {scores_filtered}')

# Plot the original test data
plt.scatter(test_labels, test_preds, alpha=0.5)

# Loop through the unique cluster labels and plot the data points for each cluster with a different color
for label in np.unique(cluster_labels):
    filtered_data = test_data[cluster_labels == label]
    plt.scatter(filtered_data[:, 1], filtered_data[:, 0], alpha=0.5, label=f'Cluster {label}')

# Add a legend, axis labels, and a title
plt.legend()
plt.xlabel('Actual')
plt.ylabel('Predicted')
plt.title('Test Predictions vs Actuals')

# Show the plot
plt.show()

Save the model trained up to 2023 on "in sample" data using cross validation

In [ ]:

filename = 'models/model_upto_2023_rolling.joblib'
dump(final_pipeline, filename)

### Load the model from storage and train on out of sample data

### Walk Forward Cross Validation on Out of sample data
- Load the model
- Preprocess the data
- Create Cross Validations for training and testing on newly seen data
- Train the model
- Make predictions
- Test and evaluate the model

In [ ]:
# filename = 'models/model_upto_2023_rolling.joblib'
# # Load the model from the .joblib file
# final_pipeline = load(filename) 

# Create your cross validation splits
cv_splitter_oos, cv_oos = create_rolling_cv_with_gap(Xoos, length=windowsize, split=split, gap=periods_future, set_labels=["train", "test"])

# Train and cross-validate
final_pipeline, oos_test_labels, oos_test_preds = cross_validate_and_train(final_pipeline, Xoos, yoos, cv_splitter_oos, model_name="Out-Of-Sample", verbose_interval=10)

# Evaluate
mse, rmse, mae, r2 = evaluate_predictions(oos_test_labels, oos_test_preds, model_name="Out-Of-Sample")

# plot the scatterplot
plot_prediction_vs_actual(oos_test_preds, oos_test_labels, title='Out-Of-Sample Predictions vs Actual Results')
extract_feature_importance(final_pipeline, X)


In [ ]:

oos_retraining_pf = vbt.Portfolio.from_signals(
    outofsample_data.close[oos_test_preds.index], # use only the test set
    entries         = oos_test_preds > 0.005, # long entry when prediction is greater than X%
    exits           = oos_test_preds < 0.00, # exit long when prediction is negative
    short_entries   = oos_test_preds < -0.005, # enter short when prediction is less than -X%
    short_exits     = oos_test_preds > 0.0, # exit short when prediction is positive
    # direction="both" # long and short
)
print(oos_retraining_pf.stats())
oos_retraining_pf.plot().show()



In [ ]:
oos_retraining_pf.trades.records_readable

In [ ]:
# Save the model
total_filename = 'models/out_of_sample_rolling.joblib'
dump(final_pipeline, total_filename)

### Simulate a portfolio in 2023 with retraining the model every 200 bars

In [ ]:
oos_test_preds.index

In [ ]:
fig = outofsample_data.close[oos_test_preds.index].vbt.plot()
outofsample_data.close.vbt.plot(fig=fig).show()
data.close.vbt.overlay_with_heatmap(test_preds).show()

In [ ]:
# insample_pf.orders.records_readable   

In [ ]:
# insample_pf.trades.records_readable

# Combine insample with out of sample

In [ ]:
fig = insample_pf.cumulative_returns.vbt.plot(trace_kwargs=dict(name='Insample')) # plot the in sample equity curve from test data not trained data
oos = insample_pf.cumulative_returns[-1] *(1+ oos_retraining_pf.returns).cumprod() # append the out of sample equity curve to the in sample equity curve
# Add the out of sample equity curve to the plot
oos.vbt.plot(fig=fig, trace_kwargs=dict(name='Out of Sample'))
normalized_price = data.close/data.close[0]
oos_normalized_price = outofsample_data.close/outofsample_data.close[0] * normalized_price[-1] # normalize the out of sample data to the last price of the in sample data
normalized_price.rename('Normalized Price').vbt.plot(fig=fig)
oos_normalized_price.rename('Out of Sample Normalized Price').vbt.plot(fig=fig)
# The gap is the warmup period for the new model to start making predictions

## Save everything to the models folder for later analysis

In [ ]:
test_preds_vs_actuals = pd.concat([test_preds, test_labels], axis=1)
test_preds_vs_actuals.columns = ['Predictions', 'Actuals']
oos_test_preds_vs_actuals = pd.concat([oos_test_preds, oos_test_labels], axis=1)
oos_test_preds_vs_actuals.columns = ['Predictions', 'Actuals']

In [ ]:

oos_test_preds_vs_actuals.tail(50)

In [ ]:
insample_pf.save('models/insample_test_portfolio_rolling.pkl')
insample_pf.stats().to_csv('models/insample_stats_test_rolling.csv')
insample_pf.trades.records_readable.to_csv('models/insample_trades_test_rolling.csv')
X.to_csv('models/insample_X_test_rolling.csv')
y.to_csv('models/insample_y_test_rolling.csv')
Xoos.to_csv('models/oos_X_test_rolling.csv')
yoos.to_csv('models/oos_y_test_rolling.csv')
test_preds_vs_actuals.to_csv('models/insample_preds_vs_actuals_test_rolling.csv')
oos_test_preds_vs_actuals.to_csv('models/oos_preds_vs_actuals_test_rolling.csv')
oos_retraining_pf.save('models/oos_retrained_portfolio_rolling.pkl')
oos_retraining_pf.stats().to_csv('models/oos_retrained_stats_rolling.csv')
oos_retraining_pf.trades.records_readable.to_csv('models/oos_retrained_trades_rolling.csv')

# Explore which features are impacting the model

In [ ]:

        
extract_feature_importance(final_pipeline, X)



A lot to unpack up above. Why are the feature scores so much different than the fscores of the features?

# Hyperparameter Tuning

### Randomized Search

In [ ]:
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import uniform, randint

# Specify hyperparameters to tune and their respective distributions
param_dist = {
    'model__learning_rate': uniform(0.01, 0.2),
    'model__n_estimators': randint(100, 1000),
    'model__max_depth': randint(3, 10),
    'model__min_child_weight': randint(1, 10),
    'model__subsample': uniform(0.5, 0.5),
    'model__colsample_bytree': uniform(0.5, 0.5),
    # add other parameters here
}

# Perform randomized search
random_search = RandomizedSearchCV(pipeline, param_dist, n_iter=10, cv=cv, scoring="neg_mean_squared_error", n_jobs=-1, verbose=10, random_state=42)
random_search.fit(X, y)

# Best parameters and score from random search
print(f"Best parameters: {random_search.best_params_}")
print(f"Best score: {random_search.best_score_}")


In [ ]:
cv_splitter, cv = create_rolling_cv_with_gap(X, length=windowsize, split=split, gap=gap, set_labels=['train', 'test'])

# The slices are obtained by using your cv_splitter on your X and y.
X_slices = cv_splitter.take(X)
y_slices = cv_splitter.take(y)

# Fit and predict with the best estimator
test_labels = []
test_preds = []
for split in X_slices.index.unique(level="split"):  
    X_train_slice = X_slices[(split, "train")]  
    y_train_slice = y_slices[(split, "train")]
    X_test_slice = X_slices[(split, "test")]
    y_test_slice = y_slices[(split, "test")]

    slice_pipeline = random_search.best_estimator_.fit(X_train_slice, y_train_slice)  # uses the best estimator from the random search
    test_pred = slice_pipeline.predict(X_test_slice)  
    test_pred = pd.Series(test_pred, index=y_test_slice.index)
    test_labels.append(y_test_slice)
    test_preds.append(test_pred)
    print(f"MSE for split {split}: {mean_squared_error(y_test_slice, test_pred)}")


test_labels = pd.concat(test_labels).rename("labels")  
test_preds = pd.concat(test_preds).rename("preds")

# Show the accuracy of the predictions
# Assuming test_labels and test_preds are your true and predicted values
mse = mean_squared_error(test_labels, test_preds)
rmse = np.sqrt(mse)  # or use mean_squared_error with squared=False
mae = mean_absolute_error(test_labels, test_preds)
r2 = r2_score(test_labels, test_preds)

print(f"Mean Squared Error (MSE): {mse}")
print(f"Root Mean Squared Error (RMSE): {rmse}")
print(f"Mean Absolute Error (MAE): {mae}")
print(f"R-squared: {r2}")

# Visualize the predictions as a heatmap plotted against the price
# data.close.vbt.overlay_with_heatmap(test_preds).show_svg()

In [ ]:
# Save the model with the best parameters
import json

# Save the model with the best parameters
dump(random_search.best_estimator_, 'models/xgboost_best_estimator_rolling.joblib')

# Save best params dictionary 
with open('models/xgboost_best_params_rolling.json', 'w') as fp:
    json.dump(random_search.best_params_, fp)


In [ ]:

hyperopt_pf = vbt.Portfolio.from_signals(
    data.close[test_preds.index],  # use only the test set
    entries         = test_preds > 0.04,  # long when probability of price increase is greater than 2%
    exits           = test_preds < 0.00,  # long when probability of price increase is greater than 2%
    short_entries   = test_preds < -0.04,  # long when probability of price increase is greater than 2%
    short_exits     = test_preds > 0.0,  # short when probability prediction is less than -5%
    # direction="both" # long and short
)
print(hyperopt_pf.stats())
hyperopt_pf.plot().show()

In [ ]:
hyperopt_pf.save('models/hyperopt_portfolio_rolling.pkl')
hyperopt_pf.trades.records_readable.to_csv('models/hyperopt_trades_rolling.csv')
hyperopt_pf.orders.records_readable.to_csv('models/hyperopt_orders_rolling.csv')
test_preds.to_csv('models/hyperopt_preds_rolling.csv')


### Grid Search Method
#### DONT RUN WITHOUT GPU

In [ ]:
# from sklearn.model_selection import GridSearchCV

# # Specify hyperparameters to tune and their respective ranges
# param_grid = {
#     'model__learning_rate': [0.01, 0.1, 0.2],
#     'model__n_estimators': [100, 500, 1000],
#     'model__max_depth': [3, 5, 7],
#     'model__min_child_weight': [1, 5, 10],
#     'model__subsample': [0.5, 0.7, 1.0],
#     'model__colsample_bytree': [0.5, 0.7, 1.0]
#     # add other parameters here
# }

# # Perform grid search
# grid_search = GridSearchCV(pipeline, param_grid, cv=cv, scoring="r2", n_jobs=-1, verbose=10)
# grid_search.fit(X, y)

# # Best parameters and score from grid search
# print(f"Best parameters: {grid_search.best_params_}")
# print(f"Best score: {grid_search.best_score_}")


In [ ]:
# cv_splitter, cv = create_rolling_cv(X, length=200, split=.9)

# # The slices are obtained by using your cv_splitter on your X and y.
# X_slices = cv_splitter.take(X)
# y_slices = cv_splitter.take(y)

# # Here, we train the model using the slices and the best estimator from your RandomizedSearchCV
# test_labels, test_preds, final_pipeline = cross_validate_and_train(random_search.best_estimator_, X_slices, y_slices, cv_splitter, model_name="Random Search Best Estimator")

# # And now we evaluate the predictions.
# mse, rmse, mae, r2 = evaluate_predictions(test_labels, test_preds, model_name="Random Search Best Estimator")


In [ ]:
# grid_pf = vbt.Portfolio.from_signals(
#     data.close[test_preds.index], # use only the test set
#     entries         = test_preds > 0.05, # long when prediction > X%
#     exits           = test_preds < 0.00, # exit when prediction is negative
#     short_entries   = test_preds < -0.05, # short when prediction < -X%
#     short_exits     = test_preds > 0.00, # exit when prediction is positive
# )
# print(grid_pf.stats())
# grid_pf.plot().show()
